In [ ]:
import numpy as np

# Combine features

feature_list = [feature_vec1, feature_vec2, ...]
# Create an array stack, NOTE: StandardScaler() expects np.float64
X = np.vstack(feature_list).astype(np.float64)


In [ ]:
from sklearn.preprocessing import StandardScaler
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

In [42]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
import glob
import os

In [4]:
# cd to helper function dir
current_dir = os.getcwd()
path = os.path.abspath(os.path.join(current_dir, ".."))
os.chdir(path)

from helper_functions import *

# cd back to working dir
current_dir = os.getcwd()
path = os.path.abspath(os.path.join(current_dir, "code-snippets"))
os.chdir(path)

In [52]:
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(image)      
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        rhist, ghist, bhist, bin_centers, hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features)))
    # Return list of feature vectors
    return features



# imgs = ['/Users/andrew/Documents/udacity/car1/proj5-vehicle-detection/test_images/extra 3.png']
# feats = extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
#                         hist_bins=32, hist_range=(0, 256))

# feats

In [48]:
def extractFeaturesNormalise(image_list, colour_space = 'RGB', size=(32, 32) ):
    features = extract_features(image_list, cspace=colour_space, spatial_size=size,
                        hist_bins=32, hist_range=(0, 256))
    scaled_X = []
    if len(features) > 0:                      
        # Fit a per-column scaler
        X_scaler = StandardScaler().fit(X)
        # Apply the scaler to X
        scaled_X = X_scaler.transform(X)
    
    return scaled_X

In [63]:
images = glob.glob('../test_images/*.png')

cars = []
notcars = []
for image in images:
    if 'extra' in image:
    #if 'image' in image or 'extra' in image:
        notcars.append(image)
    else:
        cars.append(image)
        
car_features = extract_features(cars, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256))
notcar_features = extract_features(notcars, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256))

scaled = extractFeaturesNormalise(cars)
print(scaled)

if len(car_features) > 0:
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    
    print(len(scaled_X))
    y_train = [1,1,1,1,0,0,0,0]
    
    
    car_ind = np.random.randint(0, len(cars))
    # Plot an example of raw and scaled features
    fig = plt.figure(figsize=(12,4))
    plt.subplot(131)
    plt.imshow(mpimg.imread(cars[car_ind]))
    plt.title('Original Image')
    plt.subplot(132)
    plt.plot(X[car_ind])
    plt.title('Raw Features')
    plt.subplot(133)
    plt.plot(scaled_X[car_ind])
    plt.title('Normalized Features')
    fig.tight_layout()
else: 
    print('Your function only returns empty feature vectors...')
    


[[ 0.3461896   1.16728644  0.8471775  ...,  0.          0.          0.        ]
 [-0.67303956 -0.79055164 -0.58573123 ...,  0.          0.          0.        ]
 [ 1.04316952  1.64360825  2.07372315 ...,  0.          0.          0.        ]
 ..., 
 [ 0.51316364 -0.0803625  -0.25023079 ...,  0.          0.          0.        ]
 [ 1.00951836  0.66235463  0.52305672 ...,  0.          0.          0.        ]
 [ 0.36173369 -0.85560714 -0.98669512 ...,  0.          0.          0.        ]]
8
[1, 1, 1, 1, 0, 0, 0, 0]


## Gridsearch

In [70]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svr = svm.SVC()
clf = GridSearchCV(svr, parameters)
clf.fit(scaled_X, y_train)

clf.predict(scaled_X)

array([1, 1, 1, 1, 0, 0, 0, 0])

## Random Search

In [74]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from time import time

# specify parameters and distributions to sample from
param_dist = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

# run randomized search
n_iter_search = 2
svr = svm.SVC()
random_search = RandomizedSearchCV(svr, param_dist, n_iter=n_iter_search)
start = time()
random_search.fit(scaled_X, y_train)

random_search.predict(scaled_X)



array([1, 1, 1, 1, 0, 0, 0, 0])